In [1]:
import os
import cv2
from glob import glob

from tqdm import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

In [2]:
DATA_PATH = 'data'
train_dir = os.path.join(DATA_PATH, 'train')
valid_dir = os.path.join(DATA_PATH, 'val')

In [3]:
train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
    ]))

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64)

In [5]:
valid_dataset = datasets.ImageFolder(
    valid_dir,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
    ]))

In [6]:
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=64)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
from torchvision.models.resnet import resnet50
model=resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
print(model.fc)
# num_fc_ftr = model.fc.in_features #获取到fc层的输入
# model.fc = nn.Linear(num_fc_ftr, len(breeds)) # 定义一个新的FC层
# model=model.to(device)

model.fc=nn.Linear(2048, 196)
model.to(device)

Linear(in_features=2048, out_features=1000, bias=True)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, 0.9)

In [10]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam([
#     {'params':model.fc.parameters()}
# ], lr=0.001)

In [11]:
def train(model,device, train_loader, epoch):
    model.train()
    for data in tqdm(train_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))

In [12]:
def valid(model, device, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for data in tqdm(valid_loader):          
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            valid_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    valid_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(valid_dataset), 100. * correct / len(valid_dataset)))

In [13]:
# def test(model, device, valid_loader):
#     model.eval()
#     valid_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for i,data in enumerate(valid_loader):          
#             x,y= data
#             x=x.to(device)
#             y=y.to(device)
#             optimizer.zero_grad()
#             y_hat = model(x)
#             valid_loss += criterion(y_hat, y).item() # sum up batch loss
#             pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
#             correct += pred.eq(y.view_as(pred)).sum().item()
#     valid_loss /= len(valid_loader.dataset)
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         valid_loss, correct, len(valid_dataset),
#         100. * correct / len(valid_dataset)))

In [14]:
for epoch in range(1, 1000):
    train(model=model, device=device, train_loader=train_loader, epoch=epoch)
    valid(model=model, device=device, valid_loader=valid_loader)

  0%|          | 0/102 [00:00<?, ?it/s]/home/xiaolong/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/26 [00:00<?, ?it/s]

Train Epoch: 1	 Loss: 13.887687


  0%|          | 0/102 [00:00<?, ?it/s]


Test set: Average loss: 0.2056, Accuracy: 16/1628 (1%)



  0%|          | 0/26 [00:00<?, ?it/s]

...


Test set: Average loss: 0.1441, Accuracy: 306/1628 (19%)



  0%|          | 0/26 [00:00<?, ?it/s]

Train Epoch: 999	 Loss: 4.773308


100%|██████████| 26/26 [00:17<00:00,  1.48it/s]


Test set: Average loss: 0.1417, Accuracy: 295/1628 (18%)

